# Jupyter Dashのデモ（plotly.expressで分布グラフの表示）

In [17]:
# japanize_matplotlibが未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install japanize-matplotlib

In [18]:
# JupyterDash他が未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install jupyter_dash
#!pip install --upgrade plotly

In [19]:
# Googleドライブがマウントされていない場合に、2行のコメント解除して本セルを単独で実行する
#from google.colab import drive
#drive.mount('/content/drive')

In [20]:
# matplotlibで日本語を使えるようにする
import japanize_matplotlib

In [21]:
import pandas as pd
import numpy as np

In [22]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from dash import html
from dash import dcc
from dash import dash_table as dt
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
from dash_table import DataTable, FormatTemplate
from dash_table.Format import Format

## データファイルを読み込む

In [23]:
 df1 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/都道府県各種指標.xlsx',sheet_name='data')
 df1 = df1.assign(ch_year = df1['年度'].apply(lambda x: str(x)))
 df1.sort_values('年度',ascending=False,inplace=True)
 df1.head(3)

,年度,地域コード,都道府県,総人口,総人口（男）,総人口（女）,日本人人口,日本人人口（男）,日本人人口（女）,15歳未満人口,15歳未満人口（男）,15歳未満人口（女）,15～64歳人口,15～64歳人口（男）,15～64歳人口（女）,65歳以上人口,65歳以上人口（男）,65歳以上人口（女）,出生数,出生数（男）,出生数（女）,合計特殊出生率,死亡数,死亡数（男）,死亡数（女）,転入者数（日本人移動者）,転入者数（日本人移動者）（男）,転入者数（日本人移動者）（女）,転出者数（日本人移動者）,転出者数（日本人移動者）（男）,転出者数（日本人移動者）（女）,婚姻件数,離婚件数,年平均気温,最高気温（日最高気温の月平均の最高値）,最低気温（日最低気温の月平均の最低値）,降水日数（年間）,降水量（年間）,着工建築物数,着工建築物床面積,...,大学卒業者のうち進学者数,専修学校数,各種学校数,専修学校生徒数,各種学校生徒数,新規求職申込件数（一般）,月間有効求職者数（一般）,月間有効求人数（一般）,充足数（一般）,就職件数（一般）,県内就職件数（一般）,一般旅券発行件数,着工新設住宅戸数,着工新設持家数,着工新設貸家数,着工新設分譲住宅数,着工新設住宅床面積,着工新設持家床面積,着工新設分譲住宅床面積,着工新設貸家床面積,一般病院数,一般診療所数,歯科診療所数,保育所等数,保育所等定員数,保育所等利用待機児童数,保育所等在所児数,保育所等保育士数,消費支出（二人以上の世帯）,食料費（二人以上の世帯）,住居費（二人以上の世帯）,光熱・水道費（二人以上の世帯）,家具・家事用品費（二人以上の世帯）,被服及び履物費（二人以上の世帯）,保健医療費（二人以上の世帯）,交通・通信費（二人以上の世帯）,教育費（二人以上の世帯）,教養娯楽費（二人以上の世帯）,その他の消費支出（二人以上の世帯）,ch_year
0,2018,R01000,北海道,5286000,2489000,2797000,5253000,2475000,2778000,577000,295000,282000,3052000,1502000,1551000,1656000,692000,964000,32642,16681,15961,1.27,64187,32757,31430,47269,28263,19006,55222,31407,23815,22916,9971,9.5,25.7,-7.6,157,1282.0,21437,5192636,...,2112,168,49,25820,3780,200363,734530,870195,43961,44510,42819,111793,35761,11740,19535,4067,2934844,1445452,415385,1041957,486,3392,2905,1003,80564,129,76931,10221,281054,69044,19089,27012,8845,11517,11437,39308,10825,26991,56984,2018
24,2018,R03000,岩手県,1241000,598000,643000,1234000,595000,639000,140000,72000,68000,697000,356000,341000,403000,170000,234000,7615,3853,3762,1.41,17390,8657,8733,16285,9491,6794,21285,11408,9877,4439,1843,11.0,29.4,-6.9,120,1322.0,8093,1658999,...,375,36,7,5021,416,47324,168658,233200,18590,19706,17578,17700,8496,4165,3542,704,749949,501126,71321,172101,78,882,583,379,30264,145,28639,3731,286710,75110,11459,25872,9274,10551,12318,48015,7923,25982,60205,2018
264,2018,R23000,愛知県,7537000,3770000,3767000,7323000,3665000,3658000,1002000,513000,489000,4659000,2420000,2239000,1875000,837000,1039000,61230,31259,29971,1.54,68833,36713,32120,111067,65278,45789,108908,63713,45195,39117,12653,16.9,35.3,0.2,111,1695.5,43337,9522617,...,3470,176,69,47452,12465,140162,618325,1181650,36150,34272,30870,280253,68801,19757,26690,21376,5802739,2466207,2020600,1270604,286,5404,3737,1438,181705,238,154752,17385,286146,75866,16509,20580,11894,11681,13760,43902,10876,29970,51110,2018


表示候補の列名のリストを作成する

In [24]:
my_list = df1.columns
to_remove = ['年度','地域コード','都道府県']
remained_list = [i for i in my_list if i not in to_remove]

In [25]:
# 外部スタイルシートを読み込む
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# JupyterDashの「箱」を作る
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

画面に表示するコンテンツを定義する

In [26]:
# Dash表示用の設定
default_kpi = '総人口'
plot_type = ['箱ひげ図','散布図','バイオリン']
default_plot = '箱ひげ図'

In [27]:
# ドロップダウンリストを定義する
content1 = dcc.Dropdown(
    id = 'dropdown-for-kpi', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in remained_list], # 指標値の種類を指定
    value = default_kpi # デフォルト値を設定
)
content2 = dcc.Dropdown(
    id = 'dropdown-for-plot', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in plot_type], # グラフのタイプを指定
    value = default_plot # デフォルト値を設定
)
content3 = dcc.Graph(id='box-plot')

In [28]:
# 画面のレイアウトを定義する
app.layout = html.Div([
                       html.Div(content1,
                                style={'width':'20%','display':'inline-block','margin-right':10}),
                       html.Div(content2,
                                style={'width':'20%','display':'inline-block','margin-right':10}),
                       content3
])

In [29]:
# コールバックを定義する（ドロップダウンリストで選択された値をもとに図を描画する）
# どこから入力し、どこへ出力するかを指定する
@app.callback(
    Output(component_id='box-plot',component_property='figure'),
    [Input(component_id='dropdown-for-kpi',component_property='value'),
     Input(component_id='dropdown-for-plot',component_property='value')]
)
def generate_chart(inval1,inval2): # 行間をあけずに関数部分を書く（関数名は何でも良い）
  if inval2 == '箱ひげ図':
    fig = px.box(df1, x=inval1, y='ch_year', hover_name='都道府県') # 箱ひげ図を描く
    fig.update_layout(boxgroupgap=0.1,height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":20})
    fig.update_traces(width=0.6)
  elif inval2 == '散布図':
    fig = px.scatter(df1, x=inval1, y='ch_year', hover_name='都道府県') # 散布図を描く
    fig.update_layout(boxgroupgap=0.1,height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":20})
  else:
    fig = px.violin(df1, x=inval1, y='ch_year') # バイオリンプロットを描く
    fig.update_layout(boxgroupgap=0.1,height = 450,margin=dict(l=20,r=20,t=20,b=20),
                      paper_bgcolor="GhostWhite",font={"family":"Meiryo","size":20})
    fig.update_traces(width=0.6)
  return fig

In [30]:
# サーバーを起動する
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>